# Deep Convolutional Q-Learning for Pac-Man

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [1]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 9.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.0 MB/s eta 0:00:00
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446660 sha256=a7681f1c466792b2ae1501a82a80be36d7cc70cf65c07a948eb5f7be917c479c
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed

### Importing the libraries

In [2]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque
from torch.utils.data import DataLoader, TensorDataset

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [3]:
class Network(nn.Module):

  def __init__(self, action_size,seed=42):
      super(Network,self).__init__()
      self.seed=torch.manual_seed(seed)

      #Making the "eyes" of the AI
      #1st layer
      self.conv1=nn.Conv2d(3,32,kernel_size=8,stride=4)
      self.bn1=nn.BatchNorm2d(32)
      #2nd layer
      self.conv2=nn.Conv2d(32,64,kernel_size=4,stride=2)
      self.bn2=nn.BatchNorm2d(64)
      #3rd layer
      self.conv3=nn.Conv2d(64,64,kernel_size=3,stride=1)
      self.bn3=nn.BatchNorm2d(64)
      #4th layer
      self.conv4=nn.Conv2d(64,128,kernel_size=3,stride=1)
      self.bn4=nn.BatchNorm2d(128)

      #full connections
      #no of neurons in the flattening layer=(input_size - kernel_size + 2*padding)/stride +1
      self.fc1=nn.Linear(10*10*128,512)
      self.fc2=nn.Linear(512,256)
      self.fc3=nn.Linear(256,action_size)

  #forward propagation
  def forward(self,state):
      x=F.relu(self.bn1(self.conv1(state)))
      x=F.relu(self.bn2(self.conv2(x)))
      x=F.relu(self.bn3(self.conv3(x)))
      x=F.relu(self.bn4(self.conv4(x)))

      #reshape x
      x=x.view(x.size(0),-1)

      #forward
      x=F.relu(self.fc1(x))
      x=F.relu(self.fc2(x))
      return self.fc3(x)


## Part 2 - Training the AI

### Setting up the environment

In [4]:
import gymnasium as gym
env=gym.make('MsPacmanDeterministic-v0',full_action_space=False)
state_shape=env.observation_space.shape
state_size=env.observation_space.shape[0]
number_actions=env.action_space.n
print('State shape ',state_shape)
print('State size',state_size)
print('Number of actions',number_actions)

/usr/local/lib/python3.10/dist-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment MsPacmanDeterministic-v0 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(


State shape  (210, 160, 3)
State size 210
Number of actions 9


### Initializing the hyperparameters

In [5]:
learning_rate=5e-4
minibatch_size=64
discount_factor=0.99

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Preprocessing the frames

In [6]:
from PIL import Image
from torchvision import transforms

def preprocess_frame(frame):
  frame=Image.fromarray(frame)
  preprocess=transforms.Compose([transforms.Resize((128,128)),transforms.ToTensor()])
  return preprocess(frame).unsqueeze(0)


### Implementing the DCQN class

In [7]:
class Agent():

  def __init__(self, action_size):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.action_size = action_size
    self.local_qnetwork = Network(action_size).to(self.device)
    self.target_qnetwork = Network(action_size).to(self.device)
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate)
    self.memory = deque(maxlen = 10000)

  def step(self, state, action, reward, next_state, done):
    state = preprocess_frame(state)
    next_state = preprocess_frame(next_state)
    self.memory.append((state, action, reward, next_state, done))
    if len(self.memory) > minibatch_size:
      experiences = random.sample(self.memory, k = minibatch_size)
      self.learn(experiences, discount_factor)

  def act(self, state, epsilon = 0.):
    state = preprocess_frame(state).to(self.device)
    self.local_qnetwork.eval()
    with torch.no_grad():
      action_values = self.local_qnetwork(state)
    self.local_qnetwork.train()
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  def learn(self, experiences, discount_factor):
    states, actions, rewards, next_states, dones = zip(*experiences)
    states = torch.from_numpy(np.vstack(states)).float().to(self.device)
    actions = torch.from_numpy(np.vstack(actions)).long().to(self.device)
    rewards = torch.from_numpy(np.vstack(rewards)).float().to(self.device)
    next_states = torch.from_numpy(np.vstack(next_states)).float().to(self.device)
    dones = torch.from_numpy(np.vstack(dones).astype(np.uint8)).float().to(self.device)
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + discount_factor * next_q_targets * (1 - dones)
    q_expected = self.local_qnetwork(states).gather(1, actions)
    loss = F.mse_loss(q_expected, q_targets)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Initializing the DCQN agent

In [8]:
agent=Agent(number_actions)

### Training the DCQN agent

In [9]:
#hyperparameters
number_episodes=2000
maximum_number_timesteps_per_episode=10000
epsilon_starting_value=1.0
epsilon_ending_value=0.01
epsilon_decay_value=0.995
epsilon=epsilon_starting_value
scores_on_100_episodes=deque(maxlen=100)

for episode in range(1,number_episodes+1):
  #reset environment
  state,_=env.reset()
  score=0
  for t in range(maximum_number_timesteps_per_episode):
    #select action
    action=agent.act(state,epsilon)
    next_state,reward,done,_,_=env.step(action)
    agent.step(state,action,reward,next_state,done)
    state=next_state
    score+=reward
    if done:
      break
  scores_on_100_episodes.append(score)
  epsilon=max(epsilon_ending_value,epsilon_decay_value*epsilon)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode,np.mean(scores_on_100_episodes)),end="")
  if episode %100==0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode,np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes)>=500.0:
    print('\rEnvironment solved in {:d} episode!\tAverage Score: {:.2f}'.format(episode-100,np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(),'checkpoint.pth')
    break


Environment solved in -98 episode!	Average Score: 515.00


## Part 3 - Visualizing the results

In [10]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action)
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'MsPacmanDeterministic-v0')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()

/usr/local/lib/python3.10/dist-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment MsPacmanDeterministic-v0 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(
